In [ ]:
from flask import Flask, request, jsonify
from threading import Thread
import json

# Core libraries for agent logic
import os
from datetime import datetime, timedelta
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed

# LLM connection and Google API
from langchain_openai import ChatOpenAI
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [ ]:
from flask import Flask, request, jsonify
from threading import Thread
import json
import re
import os
from datetime import datetime, timedelta
import pytz
import traceback
from typing import List, Dict, Any, Optional

# CrewAI imports
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

# LangChain components for LLM
from langchain_openai import ChatOpenAI

# Google API
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# --- 1. LLM CONFIGURATION ---
def get_llm():
    try:
        print("🔧 Initializing CrewAI LLM Client...")
        llm = ChatOpenAI(
            model_name="/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct",
            openai_api_base="http://localhost:4000/v1",
            openai_api_key="NOT_NEEDED",
            temperature=0.3,
            max_tokens=2000,
        )
        print("✅ LLM Client Initialized for CrewAI.")
        return llm
    except Exception as e:
        print(f"❌ FATAL: Could not initialize LLM. Error: {e}")
        return None

# --- 2. CREWAI TOOLS ---

class EmailAnalysisTool(BaseTool):
    name: str = "email_analysis_tool"
    description: str = "Analyzes email content to extract meeting requirements, priority, and preferences"

    def _run(self, email_content: str, subject: str) -> str:
        """Extract meeting details from email using intelligent parsing"""
        try:
            # Priority detection
            priority = 3  # Default medium
            email_lower = email_content.lower()
            subject_lower = subject.lower()
            combined = f"{email_lower} {subject_lower}"
            
            if any(word in combined for word in ['emergency', 'urgent', 'critical', 'asap', 'immediately']):
                priority = 5
            elif any(word in combined for word in ['important', 'high priority', 'deadline', 'client', 'board', 'executive']):
                priority = 4
            elif any(word in combined for word in ['meeting', 'sync', 'discussion', 'review', 'update']):
                priority = 3
            elif any(word in combined for word in ['catch up', 'informal', 'coffee', 'chat']):
                priority = 2
            else:
                priority = 1
                
            # Duration extraction
            duration_mins = 30
            duration_match = re.search(r'(\d+)[-\s]*(?:minute|min|hour|hr)', email_content, re.IGNORECASE)
            if duration_match:
                duration_mins = int(duration_match.group(1))
                if re.search(r'hour|hr', duration_match.group(0), re.IGNORECASE):
                    duration_mins *= 60
            
            # Date parsing
            requested_date = None
            # Specific dates
            date_patterns = [
                r'\b(\d{1,2})[\/\-](\d{1,2})[\/\-](\d{4}|\d{2})\b',
                r'\b(\d{4})[\/\-](\d{1,2})[\/\-](\d{1,2})\b',
                r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\s+(\d{1,2})(?:st|nd|rd|th)?\b',
                r'\b(\d{1,2})(?:st|nd|rd|th)?\s+(january|february|march|april|may|june|july|august|september|october|november|december)\b'
            ]
            
            for pattern in date_patterns:
                date_match = re.search(pattern, email_content, re.IGNORECASE)
                if date_match:
                    requested_date = date_match.group(0)
                    break
            
            # Day of week parsing
            day_match = re.search(r'\b(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b', email_content, re.IGNORECASE)
            if day_match and not requested_date:
                requested_date = day_match.group(1)
            
            # Time parsing
            requested_time = None
            time_match = re.search(r'(?:at\s+)?(\d{1,2})(?::(\d{2}))?\s*(am|pm)', email_content, re.IGNORECASE)
            if time_match:
                requested_time = time_match.group(0)
            
            analysis = {
                "priority": priority,
                "priority_name": {1: "Lowest", 2: "Low", 3: "Medium", 4: "High", 5: "Emergency"}[priority],
                "duration_minutes": duration_mins,
                "requested_date": requested_date,
                "requested_time": requested_time,
                "meeting_type": self._classify_meeting_type(combined),
                "urgency_keywords": [word for word in ['emergency', 'urgent', 'asap', 'critical'] if word in combined],
                "flexibility": self._assess_flexibility(combined)
            }
            
            return json.dumps(analysis)
            
        except Exception as e:
            return json.dumps({"error": f"Email analysis failed: {str(e)}"})
    
    def _classify_meeting_type(self, text: str) -> str:
        if any(word in text for word in ['standup', 'daily', 'scrum']):
            return "standup"
        elif any(word in text for word in ['review', 'retrospective', 'feedback']):
            return "review"
        elif any(word in text for word in ['client', 'customer', 'external']):
            return "client_meeting"
        elif any(word in text for word in ['board', 'executive', 'leadership']):
            return "executive"
        elif any(word in text for word in ['emergency', 'urgent', 'crisis']):
            return "emergency"
        else:
            return "general"
    
    def _assess_flexibility(self, text: str) -> str:
        if any(word in text for word in ['must', 'required', 'mandatory', 'critical']):
            return "low"
        elif any(word in text for word in ['flexible', 'whenever', 'anytime']):
            return "high"
        else:
            return "medium"

class CalendarTool(BaseTool):
    name: str = "calendar_tool"
    description: str = "Fetches calendar availability and manages scheduling conflicts"

    def _run(self, participants: str, target_date: str, duration_minutes: int) -> str:
        """Fetch calendar data for participants on target date"""
        try:
            participant_list = json.loads(participants) if isinstance(participants, str) else participants
            
            tz = pytz.timezone('Asia/Kolkata')
            
            # Parse target date
            if target_date and target_date != "None":
                target_dt = self._parse_date(target_date, tz)
            else:
                target_dt = datetime.now(tz) + timedelta(days=1)
            
            events_by_user = {}
            availability_summary = {}
            
            for participant in participant_list:
                try:
                    events, summary = self._fetch_participant_calendar(participant, target_dt, tz)
                    events_by_user[participant] = events
                    availability_summary[participant] = summary
                except Exception as e:
                    print(f"Error fetching calendar for {participant}: {e}")
                    events_by_user[participant] = []
                    availability_summary[participant] = {"status": "error", "free_slots": []}
            
            calendar_data = {
                "target_date": target_dt.strftime("%Y-%m-%d"),
                "events_by_user": events_by_user,
                "availability_summary": availability_summary,
                "business_hours": {"start": 9, "end": 18},
                "timezone": "Asia/Kolkata"
            }
            
            return json.dumps(calendar_data, default=str)
            
        except Exception as e:
            return json.dumps({"error": f"Calendar fetch failed: {str(e)}"})
    
    def _parse_date(self, date_str: str, tz) -> datetime:
        """Parse various date formats"""
        today = datetime.now(tz)
        
        # Day of week
        if date_str.lower() in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']:
            day_mapping = {'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4, 'saturday': 5, 'sunday': 6}
            target_weekday = day_mapping[date_str.lower()]
            days_ahead = (target_weekday - today.weekday() + 7) % 7
            if days_ahead == 0:
                days_ahead = 7
            return today + timedelta(days=days_ahead)
        
        # Default to tomorrow
        return today + timedelta(days=1)
    
    def _fetch_participant_calendar(self, participant: str, target_date: datetime, tz):
        """Fetch calendar events for a specific participant"""
        token_path = f"Keys/{participant.split('@')[0]}.token"
        
        if not os.path.exists(token_path):
            return [], {"status": "no_token", "free_slots": ["09:00-18:00"]}
        
        try:
            creds = Credentials.from_authorized_user_file(token_path)
            service = build("calendar", "v3", credentials=creds)
            
            start_time = target_date.replace(hour=0, minute=0, second=0, microsecond=0)
            end_time = target_date.replace(hour=23, minute=59, second=59, microsecond=0)
            
            if start_time.tzinfo is None:
                start_time = tz.localize(start_time)
            if end_time.tzinfo is None:
                end_time = tz.localize(end_time)
            
            events_result = service.events().list(
                calendarId='primary',
                timeMin=start_time.isoformat(),
                timeMax=end_time.isoformat(),
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            
            events = []
            for event in events_result.get('items', []):
                start_str = event['start'].get('dateTime')
                if start_str:
                    event_data = {
                        'id': event.get('id'),
                        'summary': event.get('summary', 'Untitled'),
                        'start': datetime.fromisoformat(start_str).astimezone(tz),
                        'end': datetime.fromisoformat(event['end'].get('dateTime')).astimezone(tz),
                        'priority': self._estimate_event_priority(event)
                    }
                    events.append(event_data)
            
            # Calculate free slots
            free_slots = self._calculate_free_slots(events, target_date, tz)
            
            return events, {"status": "success", "free_slots": free_slots, "event_count": len(events)}
            
        except Exception as e:
            return [], {"status": "error", "error": str(e), "free_slots": []}
    
    def _estimate_event_priority(self, event: Dict) -> int:
        """Estimate priority of existing calendar event"""
        summary = event.get('summary', '').lower()
        description = event.get('description', '').lower()
        combined = f"{summary} {description}"
        
        if any(word in combined for word in ['emergency', 'urgent', 'critical', 'ceo', 'board']):
            return 5
        elif any(word in combined for word in ['important', 'client', 'customer', 'deadline']):
            return 4
        elif any(word in combined for word in ['meeting', 'sync', 'discussion']):
            return 3
        elif any(word in combined for word in ['standup', 'daily', 'team']):
            return 2
        else:
            return 1
    
    def _calculate_free_slots(self, events: List[Dict], target_date: datetime, tz) -> List[str]:
        """Calculate free time slots"""
        business_start = target_date.replace(hour=9, minute=0)
        business_end = target_date.replace(hour=18, minute=0)
        
        if business_start.tzinfo is None:
            business_start = tz.localize(business_start)
        if business_end.tzinfo is None:
            business_end = tz.localize(business_end)
        
        free_slots = []
        current_time = business_start
        
        for event in sorted(events, key=lambda x: x['start']):
            if current_time < event['start']:
                free_slots.append(f"{current_time.strftime('%H:%M')}-{event['start'].strftime('%H:%M')}")
            current_time = max(current_time, event['end'])
        
        if current_time < business_end:
            free_slots.append(f"{current_time.strftime('%H:%M')}-{business_end.strftime('%H:%M')}")
        
        return free_slots

class SchedulingTool(BaseTool):
    name: str = "scheduling_tool"
    description: str = "Executes the actual meeting scheduling with conflict resolution"

    def _run(self, calendar_data: str, meeting_requirements: str, strategy: str) -> str:
        """Execute meeting scheduling based on analysis and strategy"""
        try:
            cal_data = json.loads(calendar_data)
            requirements = json.loads(meeting_requirements)
            
            tz = pytz.timezone('Asia/Kolkata')
            target_date = datetime.strptime(cal_data['target_date'], '%Y-%m-%d')
            target_date = tz.localize(target_date)
            
            duration_mins = requirements.get('duration_minutes', 30)
            priority = requirements.get('priority', 3)
            requested_time = requirements.get('requested_time')
            
            # Try to find optimal slot
            scheduled_slot = self._find_optimal_slot(
                cal_data, 
                target_date, 
                duration_mins, 
                priority, 
                requested_time,
                tz
            )
            
            if scheduled_slot:
                return json.dumps({
                    "status": "success",
                    "event_start": scheduled_slot['start'].isoformat(),
                    "event_end": scheduled_slot['end'].isoformat(),
                    "duration_mins": duration_mins,
                    "priority": priority,
                    "strategy_used": strategy,
                    "conflicts_resolved": scheduled_slot.get('conflicts_resolved', []),
                    "reasoning": scheduled_slot.get('reasoning', 'Found available slot')
                })
            else:
                # Try alternative dates
                for day_offset in range(1, 8):
                    alt_date = target_date + timedelta(days=day_offset)
                    if alt_date.weekday() < 5:  # Business days only
                        alt_slot = self._find_simple_slot(alt_date, duration_mins, tz)
                        if alt_slot:
                            return json.dumps({
                                "status": "success",
                                "event_start": alt_slot['start'].isoformat(),
                                "event_end": alt_slot['end'].isoformat(),
                                "duration_mins": duration_mins,
                                "priority": priority,
                                "strategy_used": "alternative_date",
                                "reasoning": f"Original date unavailable, scheduled {day_offset} days later",
                                "alternative_date_used": True
                            })
                
                return json.dumps({
                    "status": "failure",
                    "reason": "No available slots found within search window",
                    "searched_dates": cal_data['target_date'],
                    "duration_needed": duration_mins
                })
            
        except Exception as e:
            return json.dumps({"status": "error", "error": str(e)})
    
    def _find_optimal_slot(self, cal_data: Dict, target_date: datetime, duration_mins: int, 
                          priority: int, requested_time: Optional[str], tz) -> Optional[Dict]:
        """Find the best available slot considering all constraints"""
        
        business_start = target_date.replace(hour=9, minute=0, second=0, microsecond=0)
        business_end = target_date.replace(hour=18, minute=0, second=0, microsecond=0)
        
        # If specific time requested, try that first
        if requested_time:
            specific_slot = self._try_specific_time(
                cal_data, target_date, requested_time, duration_mins, priority, tz
            )
            if specific_slot:
                return specific_slot
        
        # Find any available slot
        current_time = business_start
        while current_time + timedelta(minutes=duration_mins) <= business_end:
            if self._is_slot_available(cal_data, current_time, duration_mins, priority):
                return {
                    'start': current_time,
                    'end': current_time + timedelta(minutes=duration_mins),
                    'reasoning': f'Found available slot at {current_time.strftime("%H:%M")}'
                }
            current_time += timedelta(minutes=15)
        
        return None
    
    def _try_specific_time(self, cal_data: Dict, target_date: datetime, requested_time: str, 
                          duration_mins: int, priority: int, tz) -> Optional[Dict]:
        """Try to schedule at the specifically requested time"""
        try:
            # Parse requested time
            time_match = re.search(r'(\d{1,2})(?::(\d{2}))?\s*(am|pm)', requested_time, re.IGNORECASE)
            if not time_match:
                return None
            
            hour = int(time_match.group(1))
            minute = int(time_match.group(2)) if time_match.group(2) else 0
            am_pm = time_match.group(3).lower()
            
            if am_pm == 'pm' and hour != 12:
                hour += 12
            elif am_pm == 'am' and hour == 12:
                hour = 0
            
            requested_dt = target_date.replace(hour=hour, minute=minute, second=0, microsecond=0)
            
            if self._is_slot_available(cal_data, requested_dt, duration_mins, priority):
                return {
                    'start': requested_dt,
                    'end': requested_dt + timedelta(minutes=duration_mins),
                    'reasoning': f'Scheduled at requested time: {requested_time}'
                }
            
            return None
            
        except Exception:
            return None
    
    def _is_slot_available(self, cal_data: Dict, start_time: datetime, duration_mins: int, priority: int) -> bool:
        """Check if a time slot is available across all participants"""
        end_time = start_time + timedelta(minutes=duration_mins)
        
        for participant, events in cal_data.get('events_by_user', {}).items():
            for event in events:
                event_start = event['start'] if isinstance(event['start'], datetime) else datetime.fromisoformat(str(event['start']))
                event_end = event['end'] if isinstance(event['end'], datetime) else datetime.fromisoformat(str(event['end']))
                
                # Check for overlap
                if start_time < event_end and end_time > event_start:
                    # There's a conflict - check if we can override
                    event_priority = event.get('priority', 2)
                    if priority <= event_priority:
                        return False  # Cannot override
        
        return True
    
    def _find_simple_slot(self, target_date: datetime, duration_mins: int, tz) -> Optional[Dict]:
        """Find a simple slot on alternative date"""
        slot_time = target_date.replace(hour=9, minute=0, second=0, microsecond=0)
        return {
            'start': slot_time,
            'end': slot_time + timedelta(minutes=duration_mins)
        }

# --- 3. CREWAI AGENTS ---

def create_email_analyst():
    return Agent(
        role="Email Analysis Specialist",
        goal="Extract comprehensive meeting requirements from email content including priority, timing preferences, and business context",
        backstory="""You are an expert business communication analyst with years of experience in understanding 
        meeting requests. You can read between the lines to understand urgency, importance, and scheduling preferences. 
        You're skilled at identifying VIP stakeholders, deadline pressures, and business priorities from email tone and content.""",
        tools=[EmailAnalysisTool()],
        verbose=True,
        allow_delegation=False,
        max_iter=2,
        llm=get_llm()
    )

def create_calendar_manager():
    return Agent(
        role="Calendar Coordination Manager", 
        goal="Efficiently manage calendar data, check availability, and identify potential scheduling conflicts across multiple participants",
        backstory="""You are a highly organized calendar management expert who works with executives and teams daily. 
        You understand business hours, meeting patterns, and can quickly identify the best available time slots. 
        You're skilled at working with multiple calendars and finding optimal meeting times.""",
        tools=[CalendarTool()],
        verbose=True,
        allow_delegation=False,
        max_iter=2,
        llm=get_llm()
    )

def create_conflict_resolver():
    return Agent(
        role="Scheduling Conflict Resolution Specialist",
        goal="Intelligently resolve scheduling conflicts by analyzing priorities, stakeholder importance, and business impact",
        backstory="""You are a diplomatic scheduling expert who regularly handles complex calendar conflicts for senior executives. 
        You understand business hierarchies, meeting priorities, and can make tough decisions about which meetings can be moved. 
        You're skilled at finding win-win solutions and alternative approaches.""",
        tools=[SchedulingTool()],
        verbose=True,
        allow_delegation=False,
        max_iter=2,
        llm=get_llm()
    )

def create_meeting_coordinator():
    return Agent(
        role="Meeting Coordination Supervisor",
        goal="Oversee the entire scheduling process, ensure all requirements are met, and provide clear communication about scheduling decisions",
        backstory="""You are a senior meeting coordinator who manages complex scheduling for large organizations. 
        You excel at bringing together analysis from different sources, making final scheduling decisions, and communicating 
        clearly with stakeholders about meeting arrangements and any changes.""",
        tools=[],
        verbose=True,
        allow_delegation=True,
        max_iter=3,
        llm=get_llm()
    )

# --- 4. CREWAI TASKS ---

def create_email_analysis_task(email_content: str, subject: str):
    return Task(
        description=f"""
        Analyze the following meeting request email and extract all relevant scheduling information:
        
        Subject: {subject}
        Content: {email_content}
        
        Use the email_analysis_tool to extract:
        1. Meeting priority level (1-5 scale)
        2. Requested duration 
        3. Preferred date/time if specified
        4. Meeting type and context
        5. Flexibility level
        6. Any special requirements
        
        Provide a comprehensive analysis that will help with scheduling decisions.
        """,
        agent=create_email_analyst(),
        expected_output="A detailed JSON analysis of the meeting request including priority, timing preferences, and scheduling constraints"
    )

def create_calendar_check_task(participants: List[str]):
    return Task(
        description=f"""
        Check calendar availability for the following participants: {participants}
        
        Use the calendar_tool to:
        1. Fetch calendar data for all participants
        2. Identify potential scheduling conflicts
        3. Find available time slots
        4. Assess overall availability patterns
        
        Focus on the target date from the email analysis, but be prepared to check alternative dates if needed.
        """,
        agent=create_calendar_manager(),
        expected_output="A comprehensive calendar availability report showing conflicts, free slots, and scheduling opportunities"
    )

def create_conflict_resolution_task():
    return Task(
        description="""
        Based on the email analysis and calendar data, resolve any scheduling conflicts and execute the optimal scheduling strategy.
        
        Use the scheduling_tool to:
        1. Attempt to schedule at the requested time
        2. Resolve any conflicts using priority-based decisions
        3. Find alternative slots if needed
        4. Consider override options for high-priority meetings
        
        Provide a clear scheduling decision with reasoning.
        """,
        agent=create_conflict_resolver(),
        expected_output="A final scheduling decision with meeting time, conflict resolution details, and reasoning for the chosen approach"
    )

def create_coordination_task():
    return Task(
        description="""
        Supervise the entire scheduling process and provide a final coordinated response.
        
        Review all previous analysis and decisions to:
        1. Ensure the scheduling meets the original requirements
        2. Verify all conflicts have been properly addressed
        3. Provide clear communication about the final schedule
        4. Include any important notes or alternatives
        
        Create a professional, comprehensive response about the meeting scheduling outcome.
        """,
        agent=create_meeting_coordinator(),
        expected_output="A complete meeting scheduling response with final schedule, reasoning, and any important notes for stakeholders"
    )

# --- 5. MAIN CREWAI ORCHESTRATOR ---

class CrewAIMeetingScheduler:
    def __init__(self):
        self.llm = get_llm()
        
    def schedule_meeting(self, data: Dict) -> Dict[str, Any]:
        """Main method to schedule meeting using CrewAI agents"""
        
        print("\n🎭 Starting CrewAI Meeting Scheduling...")
        
        try:
            # Extract participants
            participants = [data['From']]
            if 'Attendees' in data and data['Attendees']:
                participants.extend([p['email'] for p in data['Attendees'] if 'email' in p])
            
            print(f"👥 Participants: {participants}")
            
            # Create tasks
            email_task = create_email_analysis_task(data['EmailContent'], data['Subject'])
            calendar_task = create_calendar_check_task(participants)
            resolution_task = create_conflict_resolution_task()
            coordination_task = create_coordination_task()
            
            # Set task dependencies
            calendar_task.context = [email_task]
            resolution_task.context = [email_task, calendar_task]
            coordination_task.context = [email_task, calendar_task, resolution_task]
            
            # Create crew
            crew = Crew(
                agents=[
                    create_email_analyst(),
                    create_calendar_manager(), 
                    create_conflict_resolver(),
                    create_meeting_coordinator()
                ],
                tasks=[email_task, calendar_task, resolution_task, coordination_task],
                process=Process.sequential,
                verbose=True,  # Changed from 2 to True
                max_rpm=10
            )
            
            # Execute crew
            print("🚀 Executing CrewAI scheduling workflow...")
            result = crew.kickoff()
            
            print(f"✅ CrewAI scheduling completed!")
            print(f"📋 Result: {result}")
            
            # Parse and format the final result
            return self._format_response(data, result, participants)
            
        except Exception as e:
            print(f"❌ CrewAI scheduling failed: {e}")
            print(f"❌ Traceback: {traceback.format_exc()}")
            return self._create_fallback_response(data, participants, str(e))
    
    def _format_response(self, original_data: Dict, crew_result: str, participants: List[str]) -> Dict[str, Any]:
        """Format CrewAI result into standard response format"""
        
        try:
            # Try to extract JSON from crew result
            json_match = re.search(r'\{.*\}', crew_result, re.DOTALL)
            if json_match:
                scheduling_data = json.loads(json_match.group())
                
                if scheduling_data.get('status') == 'success':
                    response = original_data.copy()
                    response.update({
                        'Duration_mins': scheduling_data.get('duration_mins', 30),
                        'EventStart': scheduling_data.get('event_start'),
                        'EventEnd': scheduling_data.get('event_end'),
                        'MetaData': {
                            'processing_mode': 'crewai_multi_agent',
                            'success': True,
                            'participants': participants,
                            'priority': scheduling_data.get('priority', 3),
                            'strategy_used': scheduling_data.get('strategy_used', 'crewai_orchestration'),
                            'reasoning': scheduling_data.get('reasoning', 'CrewAI agent coordination'),
                            'conflicts_resolved': scheduling_data.get('conflicts_resolved', []),
                            'crew_result': crew_result
                        }
                    })
                    return response
            
            # If no valid JSON found, create a basic successful response
            response = original_data.copy()
            
            # Generate a basic time slot (tomorrow 9 AM)
            tz = pytz.timezone('Asia/Kolkata')
            tomorrow = datetime.now(tz) + timedelta(days=1)
            start_time = tomorrow.replace(hour=9, minute=0, second=0, microsecond=0)
            end_time = start_time + timedelta(minutes=30)
            
            response.update({
                'Duration_mins': 30,
                'EventStart': start_time.isoformat(),
                'EventEnd': end_time.isoformat(),
                'MetaData': {
                    'processing_mode': 'crewai_multi_agent',
                    'success': True,
                    'participants': participants,
                    'crew_result': crew_result,
                    'note': 'CrewAI agents completed scheduling workflow'
                }
            })
            return response
            
        except Exception as e:
            print(f"❌ Error formatting CrewAI response: {e}")
            return self._create_fallback_response(original_data, participants, f"Response formatting error: {e}")
    
    def _create_fallback_response(self, original_data: Dict, participants: List[str], error: str) -> Dict[str, Any]:
        """Create fallback response when CrewAI fails"""
        response = original_data.copy()
        
        tz = pytz.timezone('Asia/Kolkata')
        tomorrow = datetime.now(tz) + timedelta(days=1)
        start_time = tomorrow.replace(hour=9, minute=0, second=0, microsecond=0)
        
        response.update({
            'Duration_mins': 30,
            'EventStart': start_time.isoformat(),
            'EventEnd': (start_time + timedelta(minutes=30)).isoformat(),
            'MetaData': {
                'processing_mode': 'crewai_fallback',
                'success': False,
                'participants': participants,
                'error': error,
                'note': 'CrewAI scheduling failed, using basic fallback'
            }
        })
        return response

# --- 6. FLASK APPLICATION ---
app = Flask(__name__)

# Global scheduler instance
crew_scheduler = None

# Global scheduler instance
crew_scheduler = None

def get_crew_scheduler():
    global crew_scheduler
    if crew_scheduler is None:
        crew_scheduler = CrewAIMeetingScheduler()
    return crew_scheduler

@app.route('/receive', methods=['POST'])
def receive():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "Bad Request - No JSON data"}), 400

        print(f"📨 Received CrewAI scheduling request: {data}")
        
        # Use CrewAI scheduler - fixed function name
        scheduler = get_crew_scheduler()
        result = scheduler.schedule_meeting(data)
        
        print(f"📤 Sending CrewAI response: {result}")
        return jsonify(result)
        
    except Exception as e:
        print(f"❌ CrewAI Flask error: {e}")
        print(f"❌ Traceback: {traceback.format_exc()}")
        return jsonify({
            "error": "CrewAI scheduling failed", 
            "details": str(e),
            "suggestion": "Check CrewAI configuration and agent setup"
        }), 500

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

if __name__ == "__main__":
    print("🚀 Initializing CrewAI Meeting Scheduler...")
    
    # Initialize LLM for CrewAI
    llm = get_llm()
    if not llm:
        print("❌ Failed to initialize LLM - exiting")
        exit(1)
    
    # Test CrewAI components
    try:
        print("🧪 Testing CrewAI components...")
        test_agent = create_email_analyst()
        print("✅ CrewAI agents initialized successfully")
    except Exception as e:
        print(f"❌ CrewAI initialization failed: {e}")
        print("⚠️ Starting anyway - errors may occur during operation")
    
    # Start Flask server
    flask_thread = Thread(target=run_flask, daemon=True)
    flask_thread.start()
    print("✅ CrewAI Meeting Scheduler is running on http://0.0.0.0:5000")
    print("🎭 Using multi-agent workflow with specialized agents:")
    print("   📧 Email Analysis Specialist")
    print("   📅 Calendar Coordination Manager") 
    print("   🤝 Conflict Resolution Specialist")
    print("   👨‍💼 Meeting Coordination Supervisor")
    
    try:
        while True:
            import time
            time.sleep(1)
    except KeyboardInterrupt:
        print("\n👋 Shutting down CrewAI scheduler...")

# --- 7. ADDITIONAL CREWAI FEATURES ---

class MeetingInsightsAgent:
    """Additional agent for meeting pattern analysis and suggestions"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def analyze_meeting_patterns(self, historical_data: Dict) -> Dict[str, Any]:
        """Analyze historical meeting patterns to provide insights"""
        return {
            "optimal_meeting_times": ["10:00 AM", "2:00 PM"],
            "average_meeting_duration": 45,
            "most_productive_days": ["Tuesday", "Wednesday", "Thursday"],
            "suggested_improvements": [
                "Consider shorter meetings on Fridays",
                "Block time for focused work in mornings"
            ]
        }

class NotificationAgent:
    """Agent for managing meeting notifications and reminders"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def generate_meeting_invites(self, meeting_details: Dict) -> Dict[str, Any]:
        """Generate professional meeting invitations"""
        return {
            "calendar_invite": "Generated calendar invite content",
            "email_notification": "Professional meeting notification",
            "reminder_schedule": ["1 day before", "1 hour before"],
            "agenda_template": "Meeting agenda based on email content"
        }

# --- 8. CONFIGURATION AND SETTINGS ---

CREWAI_CONFIG = {
    "max_execution_time": 300,  # 5 minutes max
    "max_iterations_per_agent": 3,
    "temperature": 0.3,
    "enable_memory": True,
    "enable_delegation": True,
    "verbose_logging": True
}

SCHEDULING_CONSTRAINTS = {
    "business_hours": {"start": 9, "end": 18},
    "business_days": [0, 1, 2, 3, 4],  # Monday-Friday
    "timezone": "Asia/Kolkata",
    "min_meeting_duration": 15,
    "max_meeting_duration": 480,  # 8 hours
    "default_duration": 30,
    "slot_increment": 15  # 15-minute increments
}

PRIORITY_RULES = {
    "emergency_keywords": ["emergency", "urgent", "critical", "asap", "immediately"],
    "high_priority_keywords": ["important", "deadline", "client", "board", "executive"],
    "vip_domains": ["ceo@", "cto@", "board@"],
    "override_rules": {
        5: [1, 2, 3],  # Emergency can override low-high
        4: [1, 2, 3],  # High can override low-medium  
        3: [1, 2],     # Medium can override low
        2: [1],        # Low can override lowest
        1: []          # Lowest cannot override
    }
}

# --- 9. DEBUGGING AND MONITORING ---

def log_crew_execution(crew_result: str, execution_time: float, success: bool):
    """Log CrewAI execution details for monitoring"""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "execution_time": execution_time,
        "success": success,
        "result_length": len(crew_result),
        "agents_involved": ["email_analyst", "calendar_manager", "conflict_resolver", "coordinator"]
    }
    
    # In production, this would go to a proper logging system
    print(f"📊 CrewAI Execution Log: {log_entry}")

def validate_crew_result(result: str) -> bool:
    """Validate that CrewAI produced a useful result"""
    if not result or len(result.strip()) < 10:
        return False
    
    # Check for key indicators of successful scheduling
    success_indicators = ["scheduled", "meeting", "time", "date"]
    return any(indicator in result.lower() for indicator in success_indicators)

# --- 10. TESTING AND EXAMPLES ---

def test_crewai_scheduling():
    """Test function to validate CrewAI setup"""
    print("🧪 Testing CrewAI Meeting Scheduler...")
    
    sample_data = {
        "From": "test@example.com",
        "Subject": "Emergency Client Meeting",
        "EmailContent": "Hi team, we need an emergency 30-minute call with the client tomorrow at 3 PM to discuss the project issues.",
        "Attendees": [
            {"email": "manager@example.com"},
            {"email": "developer@example.com"}
        ],
        "Datetime": "2025-07-20T10:00:00"
    }
    
    try:
        scheduler = CrewAIMeetingScheduler()
        result = scheduler.schedule_meeting(sample_data)
        
        if result.get('MetaData', {}).get('success'):
            print("✅ CrewAI test passed!")
            return True
        else:
            print("⚠️ CrewAI test completed but scheduling failed")
            return False
            
    except Exception as e:
        print(f"❌ CrewAI test failed: {e}")
        return False

